<a href="https://colab.research.google.com/github/ludoveltz/test_github_fev25/blob/main/Daily_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np

try:
    # Lecture du fichier avec l'encodage correct
    df = pd.read_csv('tennis_articles.csv', encoding='latin1')

    # Exploration initiale
    print("\n📊 Aperçu du dataset:")
    print(f"Dimensions du dataset: {df.shape}")
    print(f"Nombre d'articles: {len(df)}")

    print("\n📑 Colonnes présentes:")
    for col in df.columns:
        non_null = df[col].count()
        dtype = df[col].dtype
        print(f"- {col}: {non_null} valeurs non-nulles, type: {dtype}")

    print("\n📝 Exemple d'article:")
    example = df.iloc[0]
    print(f"\nTitre: {example['article_title']}")
    print(f"Source: {example['source']}")
    print(f"Texte (premiers 300 caractères):\n{example['article_text'][:300]}...")

    # Nettoyage - garder uniquement article_text
    df_cleaned = df[['article_text']]

    print("\n🧹 Dataset nettoyé:")
    print(f"- Colonnes conservées: {df_cleaned.columns.tolist()}")
    print(f"- Nombre d'articles: {len(df_cleaned)}")

    # Statistiques sur les articles
    article_lengths = df_cleaned['article_text'].str.len()
    print("\n📈 Statistiques sur la longueur des articles:")
    print(f"- Longueur moyenne: {article_lengths.mean():.0f} caractères")
    print(f"- Longueur minimale: {article_lengths.min():.0f} caractères")
    print(f"- Longueur maximale: {article_lengths.max():.0f} caractères")

    # Sauvegarde du dataset nettoyé
    df_cleaned.to_csv('tennis_articles_cleaned.csv', index=False)
    print("\n💾 Dataset nettoyé sauvegardé dans 'tennis_articles_cleaned.csv'")

except Exception as e:
    print(f"❌ Erreur: {str(e)}")




📊 Aperçu du dataset:
Dimensions du dataset: (8, 4)
Nombre d'articles: 8

📑 Colonnes présentes:
- article_id: 8 valeurs non-nulles, type: int64
- article_title: 8 valeurs non-nulles, type: object
- article_text: 8 valeurs non-nulles, type: object
- source: 8 valeurs non-nulles, type: object

📝 Exemple d'article:

Titre: I do not have friends in tennis, says Maria Sharapova
Source: https://www.tennisworldusa.org/tennis/news/Maria_Sharapova/62220/i-do-not-have-friends-in-tennis-says-maria-sharapova/
Texte (premiers 300 caractères):
Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on...

🧹 Dataset nettoyé:
- Colonnes conservées: ['article_text']
- Nombre d'articles: 8

📈 Statistiques sur la longueur des articles:
- Longueur moyenne: 1967 caract

In [7]:
import pandas as pd
import nltk
import numpy as np
import string
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

try:
    # Téléchargement complet des ressources NLTK nécessaires
    print("⏳ Téléchargement des ressources NLTK...")
    nltk.download('punkt_tab')
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('averaged_perceptron_tagger')
    print("✅ Ressources NLTK téléchargées")

    # Chargement du dataset nettoyé
    df = pd.read_csv('tennis_articles_cleaned.csv')

    # Fonction de nettoyage du texte
    def clean_text(text):
        # Conversion en minuscules
        text = text.lower()
        # Suppression de la ponctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Suppression des chiffres
        text = re.sub(r'\d+', '', text)
        # Suppression des espaces multiples
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    # Chargement des embeddings GloVe (utilisation du modèle 100d pour la rapidité)
    print("\n⏳ Chargement des embeddings GloVe...")
    embeddings_dict = {}
    with open('glove.6B.100d.txt', 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_dict[word] = vector
    print("✅ Embeddings chargés")

    # Prétraitement des articles
    all_sentences = []
    original_sentences = []
    stop_words = set(stopwords.words('english'))

    print("\n🔄 Traitement des articles...")
    for idx, article in enumerate(df['article_text'], 1):
        print(f"Traitement de l'article {idx}/{len(df)}...")

        try:
            # Tokenization en phrases
            sentences = sent_tokenize(article)
            original_sentences.extend(sentences)

            # Nettoyage et prétraitement de chaque phrase
            for sentence in sentences:
                # Nettoyage
                clean_sent = clean_text(sentence)
                # Tokenization en mots
                words = word_tokenize(clean_sent)
                # Suppression des stop words
                words = [word for word in words if word not in stop_words]
                # Ajout à la liste
                all_sentences.append(words)

        except Exception as e:
            print(f"⚠️ Erreur lors du traitement de l'article {idx}: {str(e)}")
            continue

    print(f"✅ Nombre total de phrases traitées: {len(all_sentences)}")

    # Vectorisation des phrases
    print("\n⏳ Vectorisation des phrases...")
    sentence_vectors = []
    for sentence in all_sentences:
        if not sentence:
            continue

        word_vectors = []
        for word in sentence:
            if word in embeddings_dict:
                word_vectors.append(embeddings_dict[word])

        if word_vectors:
            sentence_vector = np.mean(word_vectors, axis=0)
            sentence_vectors.append(sentence_vector)
        else:
            sentence_vectors.append(np.zeros(100))

    print(f"✅ Vectorisation terminée: {len(sentence_vectors)} vecteurs créés")

    # Sauvegarde des résultats pour l'étape suivante
    np.save('sentence_vectors.npy', np.array(sentence_vectors))
    with open('original_sentences.txt', 'w', encoding='utf-8') as f:
        for sent in original_sentences:
            f.write(sent + '\n')

    print("\n💾 Résultats sauvegardés pour l'étape suivante")

except Exception as e:
    print(f"❌ Erreur générale: {str(e)}")



⏳ Téléchargement des ressources NLTK...


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


✅ Ressources NLTK téléchargées

⏳ Chargement des embeddings GloVe...
✅ Embeddings chargés

🔄 Traitement des articles...
Traitement de l'article 1/8...
Traitement de l'article 2/8...
Traitement de l'article 3/8...
Traitement de l'article 4/8...
Traitement de l'article 5/8...
Traitement de l'article 6/8...
Traitement de l'article 7/8...
Traitement de l'article 8/8...
✅ Nombre total de phrases traitées: 130

⏳ Vectorisation des phrases...
✅ Vectorisation terminée: 129 vecteurs créés

💾 Résultats sauvegardés pour l'étape suivante


In [9]:
import numpy as np
import networkx as nx
from scipy.spatial.distance import cosine
import pickle

try:
    # Chargement des vecteurs
    print("⏳ Chargement des vecteurs...")
    sentence_vectors = np.load('sentence_vectors.npy')
    print(f"✅ {len(sentence_vectors)} vecteurs chargés")

    # Calcul de la matrice de similarité
    print("\n⏳ Calcul de la matrice de similarité...")
    similarity_matrix = np.zeros((len(sentence_vectors), len(sentence_vectors)))

    for i in range(len(sentence_vectors)):
        for j in range(len(sentence_vectors)):
            if i != j:
                similarity = 1 - cosine(sentence_vectors[i], sentence_vectors[j])
                similarity = max(0, similarity)
                similarity_matrix[i][j] = similarity

    print("✅ Matrice de similarité calculée")

    # Statistiques de la matrice
    print("\n📊 Statistiques de similarité:")
    print(f"- Similarité moyenne: {np.mean(similarity_matrix):.3f}")
    print(f"- Similarité maximale: {np.max(similarity_matrix):.3f}")
    print(f"- Similarité minimale: {np.min(similarity_matrix):.3f}")

    # Construction du graphe avec seuil de similarité
    print("\n⏳ Construction du graphe avec seuil...")
    threshold = 0.7  # Seuil de similarité pour créer une arête
    graph = nx.Graph()

    # Ajout des nœuds
    for i in range(len(sentence_vectors)):
        graph.add_node(i)

    # Ajout des arêtes avec poids au-dessus du seuil
    for i in range(len(sentence_vectors)):
        for j in range(i + 1, len(sentence_vectors)):
            if similarity_matrix[i][j] > threshold:
                graph.add_edge(i, j, weight=similarity_matrix[i][j])

    # Statistiques du graphe
    print("📊 Statistiques du graphe:")
    print(f"- Nombre de nœuds: {graph.number_of_nodes()}")
    print(f"- Nombre d'arêtes: {graph.number_of_edges()}")
    print(f"- Densité du graphe: {nx.density(graph):.3f}")

    # Analyse des nœuds centraux
    print("\n🔍 Analyse des nœuds centraux...")
    centrality = nx.degree_centrality(graph)
    top_nodes = sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:5]

    # Chargement des phrases originales
    with open('original_sentences.txt', 'r', encoding='utf-8') as f:
        original_sentences = f.readlines()

    print("\n📈 Top 5 des phrases les plus connectées:")
    for node_id, score in top_nodes:
        print(f"\nScore de centralité: {score:.3f}")
        print(f"Phrase: {original_sentences[node_id].strip()[:100]}...")

    # Sauvegarde des résultats
    print("\n💾 Sauvegarde des résultats...")
    np.save('similarity_matrix.npy', similarity_matrix)
    with open('sentence_graph.pkl', 'wb') as f:
        pickle.dump(graph, f)
    print("✅ Résultats sauvegardés")

    # Préparation pour l'étape suivante (PageRank)
    print("\n🔄 Le graphe est prêt pour l'application de PageRank")

except Exception as e:
    print(f"❌ Erreur: {str(e)}")



⏳ Chargement des vecteurs...
✅ 129 vecteurs chargés

⏳ Calcul de la matrice de similarité...
✅ Matrice de similarité calculée

📊 Statistiques de similarité:
- Similarité moyenne: 0.724
- Similarité maximale: 0.957
- Similarité minimale: 0.000

⏳ Construction du graphe avec seuil...
📊 Statistiques du graphe:
- Nombre de nœuds: 129
- Nombre d'arêtes: 5504
- Densité du graphe: 0.667

🔍 Analyse des nœuds centraux...

📈 Top 5 des phrases les plus connectées:

Score de centralité: 0.938
Phrase: So I'm not the one to strike up a conversation about the weather and know that in the next few minut...

Score de centralité: 0.938
Phrase: It's a very pleasant atmosphere, I'd have to say, around the locker rooms....

Score de centralité: 0.922
Phrase: Roger Federer has revealed that organisers of the re-launched and condensed Davis Cup gave him three...

Score de centralité: 0.922
Phrase: Not because hed been out on a bender or anything  those days were in the past....

Score de centralité: 0.922

In [10]:
import numpy as np
import networkx as nx
import pickle

try:
    # Chargement du graphe et des phrases originales
    print("⏳ Chargement des données...")
    with open('sentence_graph.pkl', 'rb') as f:
        graph = pickle.load(f)

    with open('original_sentences.txt', 'r', encoding='utf-8') as f:
        original_sentences = f.readlines()
    original_sentences = [s.strip() for s in original_sentences]

    print("✅ Données chargées")

    # Application de l'algorithme PageRank
    print("\n⏳ Application de PageRank...")
    pagerank_scores = nx.pagerank(graph, alpha=0.85)  # alpha est le facteur d'amortissement

    # Tri des phrases selon leur score PageRank
    ranked_sentences = [(score, idx) for idx, score in pagerank_scores.items()]
    ranked_sentences.sort(reverse=True)

    # Paramètres du résumé
    summary_size = 5  # Nombre de phrases pour le résumé

    # Extraction des phrases les plus importantes
    print(f"\n📝 Résumé généré ({summary_size} phrases les plus importantes):")
    print("\n-------------------")

    for i, (score, idx) in enumerate(ranked_sentences[:summary_size], 1):
        print(f"\n{i}. Score PageRank: {score:.3f}")
        print(f"Phrase: {original_sentences[idx]}")

    # Statistiques sur les scores PageRank
    scores = [score for score, _ in ranked_sentences]
    print("\n📊 Statistiques PageRank:")
    print(f"- Score moyen: {np.mean(scores):.3f}")
    print(f"- Score maximal: {np.max(scores):.3f}")
    print(f"- Score minimal: {np.min(scores):.3f}")

    # Sauvegarde du résumé
    print("\n💾 Sauvegarde du résumé...")
    with open('tennis_articles_summary.txt', 'w', encoding='utf-8') as f:
        f.write("Résumé des articles de tennis\n")
        f.write("===========================\n\n")
        for i, (score, idx) in enumerate(ranked_sentences[:summary_size], 1):
            f.write(f"{i}. {original_sentences[idx]}\n\n")

    print("✅ Résumé sauvegardé dans 'tennis_articles_summary.txt'")

    # Analyse de la couverture thématique
    print("\n📈 Analyse de la distribution des phrases:")
    print(f"- Nombre total de phrases: {len(original_sentences)}")
    print(f"- Taux de compression: {(summary_size/len(original_sentences))*100:.1f}%")

except Exception as e:
    print(f"❌ Erreur: {str(e)}")


⏳ Chargement des données...
✅ Données chargées

⏳ Application de PageRank...

📝 Résumé généré (5 phrases les plus importantes):

-------------------

1. Score PageRank: 0.011
Phrase: It's a very pleasant atmosphere, I'd have to say, around the locker rooms.

2. Score PageRank: 0.011
Phrase: So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.

3. Score PageRank: 0.011
Phrase: Hed backed up his last-32 showingat Melbourne Park with a string of wins over elites including French Open champion and then world No.9 Gaston Gaudio and Roland Garros runner-up Martin Verkerk in 2004 before illness struck.

4. Score PageRank: 0.011
Phrase: Roger Federer has revealed that organisers of the re-launched and condensed Davis Cup gave him three days to decide if he would commit to the controversial competition.

5. Score PageRank: 0.011
Phrase: Not because hed been out on a bender or anything  those days w